In [16]:
import pandas as pd 
import numpy as np
import os

In [17]:

file_names = os.listdir('../data2/Anio2014/')
file_names

['abr_mo14.csv',
 'ago_mo14.csv',
 'dic_mo14.csv',
 'ene_mo14.csv',
 'feb_mo14.csv',
 'jul_mo14.csv',
 'jun_mo14.csv',
 'mar_mo14.csv',
 'may_mo14.csv',
 'nov_mo14.csv',
 'oct_mo14.csv',
 'sep_mo14.csv']

In [18]:
df_aire = pd.DataFrame()
for file_name in file_names:
    df_current = pd.read_csv('../data2/Anio2014/' + file_name, delimiter=';')
    print(file_name, df_current.shape)
    df_aire = pd.concat([df_aire, df_current], ignore_index=True)

abr_mo14.csv (4530, 56)
ago_mo14.csv (4567, 56)
dic_mo14.csv (4645, 56)
ene_mo14.csv (4681, 56)
feb_mo14.csv (4228, 56)
jul_mo14.csv (4578, 56)
jun_mo14.csv (4440, 56)
mar_mo14.csv (4614, 56)
may_mo14.csv (4614, 56)
nov_mo14.csv (4530, 56)
oct_mo14.csv (4623, 56)
sep_mo14.csv (4440, 56)


In [19]:
# 3. Filtrar por datos NO2
df_aire = df_aire[(df_aire['MAGNITUD'] == 8)]
df_aire.head()
df_aire.tail()

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
54395,28,79,60,8,28079060_8_8,2014,9,26,11.0,V,...,30.0,V,60.0,V,78.0,V,95.0,V,86.0,V
54396,28,79,60,8,28079060_8_8,2014,9,27,66.0,V,...,23.0,V,22.0,V,22.0,V,17.0,V,12.0,V
54397,28,79,60,8,28079060_8_8,2014,9,28,11.0,V,...,13.0,V,29.0,V,43.0,V,55.0,V,34.0,V
54398,28,79,60,8,28079060_8_8,2014,9,29,54.0,V,...,39.0,V,50.0,V,61.0,V,73.0,V,61.0,V
54399,28,79,60,8,28079060_8_8,2014,9,30,39.0,V,...,32.0,V,62.0,V,73.0,V,95.0,V,25.0,V


In [20]:
df_aire['fecha'] = pd.to_datetime(df_aire[['ANO','MES','DIA']].rename(columns = {'ANO': 'YEAR', 'MES': 'MONTH', 'DIA': 'DAY'}))


In [21]:
# 4. Extraer datos de las medidas tomadas y creamos un nuevo dataframe 'data' con el formato que nosotros queremos
data_columns = [ 'H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'H17', 'H18', 'H19', 'H20', 'H21', 'H22', 'H23', 'H24'] # feature
veri_columns = [ 'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24']

hora_column = np.arange(1,25) # feature label

data = pd.DataFrame(columns=['ano', 'mes', 'dia', 'hora', 'dato', 'verificado', 'estacion']) # DataFrame con datos finales
aux = pd.DataFrame(columns=['ano', 'mes', 'dia', 'hora', 'dato', 'verificado', 'estacion']) # aux de próxima fila a ser añadida

# para cada hora de cada fila, crea una nueva fila con su dato y su valor de verificado
for  (h,d,v) in zip(hora_column, data_columns, veri_columns): 
     aux['dato'] = df_aire [ data_columns[h-1] ] 
     aux['verificado'] = df_aire [ veri_columns[h-1] ] 
     aux['hora'] = h-1
     aux[['ano', 'mes', 'dia']] = df_aire[['ANO','MES','DIA']]
     aux['estacion'] = df_aire ['ESTACION']
     data = data.append(aux, sort = False, ignore_index = True)

data = data.astype({'hora': 'int32'}, copy = False)

data.head(10)

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
# Añadimos columna fecha de formato datetime
data['fecha'] = pd.to_datetime(data[['ano','mes','dia','hora']].rename(columns = {'ano': 'year', 'mes': 'month',
                                                                              'dia': 'day', 'hora': 'hour'}))

In [ ]:
data

In [ ]:
# Multi-index data frame con estacion y fecha :O
data.set_index(['estacion', 'fecha'], inplace=True)

In [ ]:
# Usamos fecha como indice para cada estacion
#data.set_index(['fecha'], inplace=True)

In [ ]:
# Ordnamos por estacion y luego por fecha (aprovechando el nuevo index)
data = data.sort_values(['estacion', 'fecha'])
data

In [ ]:
# Para cada estación, rellenamos con NaN si hay alguna fila perdida y guardamos el resultado en un nuevo dataframe que ya debería tener todas las filas
#estaciones = data['estacion'].unique()
estaciones = data.index.get_level_values('estacion').unique()
datetime_index = pd.date_range(start='2/1/2019', end='5/31/2019 23:00:00', freq='H', name='fecha')
data_nuevo = pd.DataFrame()
for estacion in estaciones:
    #data_estacion = data[data['estacion'] == estacion]
    data_estacion = data.loc[estacion]
    print(estacion, len(data_estacion))
    data_estacion = data_estacion.reindex(datetime_index)
    data_estacion.reset_index(inplace=True)
    print(estacion, len(data_estacion))
    data_estacion['estacion'] = estacion
    data_nuevo = data_nuevo.append(data_estacion)
    
print(len(data_nuevo))

In [ ]:
# Multi-index data frame con estacion y fecha :O
data_nuevo.set_index(['estacion', 'fecha'], inplace=True)

In [ ]:
# Actualizamos data con el nuevo df:
data = data_nuevo
data.info()
data

In [ ]:
# Mostramos los datos que se habían perdidos
missing_data = data[data['dato'].isna()]
print(f'Cantidad de datos no existentes (perdidos): {len(missing_data)}')
missing_data

In [ ]:
# # Rellenamos las columnas conciernientes a la fecha que faltan en esos datos perdidos excepto la columna 'dato' que ya trataremos más adelante
# dt = missing_data.index.get_level_values('fecha')
# data.loc[data['dato'].isna(),'ano'] = dt.year
# data.loc[data['dato'].isna(),'mes'] = dt.month
# data.loc[data['dato'].isna(),'dia'] = dt.day
# data.loc[data['dato'].isna(),'hora'] = dt.hour
# data = data.astype({'hora': 'int32'}, copy = False)

# #data.loc[data['dato'].isna(),['ano', 'mes', 'dia', 'hora']] = [dt.year, dt.month, dt.day, dt.hour]
# data[data['dato'].isna()]

In [ ]:
# 6. Por último, marcamos los datos no verificados como NaN
no_verificados = data.loc[data['verificado'] != 'V']
print(f'Cantidad de datos existentes pero no válidos:{len(no_verificados)}')
data.loc[data['verificado'] != 'V', ['dato']] = np.nan
data.loc[data['verificado'] != 'V']

In [ ]:
# borramos columna verificado
del data['verificado']
data.info()

In [ ]:
# Primero de todo, deshacemos el multíndice
data.reset_index(inplace=True) # Salvar los datos sin indices de estacion y fecha

In [ ]:
# 7. Añadimos columna para el día de la semana
#dates = data.index.get_level_values('fecha').to_pydatetime() # con multiindex
dates = data['fecha'].dt.to_pydatetime() # con column y sin multiindex
days_of_week = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
data['dia_semana'] = np.array([days_of_week[d.weekday()] for d in dates])

In [ ]:
data = data.rename(columns={'dato': 'NO2'}, copy = False)

In [ ]:
data.info()
data

In [ ]:
# Serie con solamente los datos de contaminación
datos = data['NO2']

In [ ]:
# Número de datos no válidos
len(datos[datos.isnull()])

In [ ]:
datos.to_csv('pre_24est-feb-may_2019.csv') # Salvado serie solo datos históricos
data.to_csv('pre_24est-feb-may_2019+date.csv') # Salvado con los datos de las fechas

In [ ]:
data_estacion = data[data['estacion'] == 4].copy(deep=True)
datos_estacion = data_estacion['NO2'].values

In [ ]:
from itertools import accumulate
import matplotlib.pyplot as plt

# Visualizamos datos de salida
range_data = range(len(datos_estacion))
plt.title('Datos de NO² por horas para el periodo seleccionado')
plt.xlabel('Horas (en secuencia)')
plt.ylabel('NO² (en µg/m³)')

month_marks = list(map(lambda x: x * 24, accumulate(days_month, lambda x, y: (x+y))))
plt.plot(range_data, datos_estacion)
for mark in month_marks:
    plt.axvline(x=mark, color='r', linestyle='--')

In [ ]:
datos_estacion = np.reshape(datos_estacion, (datos_estacion.shape[0], 1))

In [ ]:
for estacion in estaciones[1:]:
    aux = data[data['estacion'] == estacion]['NO2'].values
    aux = np.reshape(aux, (aux.shape[0], 1))
    datos_estacion = np.concatenate((datos_estacion, aux), axis=-1)

In [ ]:
fig, axs = plt.subplots(2, 1)

#ax = fig.add_subplot(111)
#plt.title('Datos de NO² por horas para el periodo seleccionado')
#plt.xlabel('Horas (en secuencia)')
#plt.ylabel('NO² (en µg/m³)')

axs[0].plot(range_data, datos_estacion[:,0])
for mark in month_marks:
    axs[0].axvline(x=mark, color='r', linestyle='--')
axs[1].plot(range_data, datos_estacion[:,1])
for mark in month_marks:
    axs[1].axvline(x=mark, color='r', linestyle='--')